# Explore here

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
# pd.read_csv('https://raw.githubusercontent.com/4GeeksAcademy/k-nearest-neighbors-project-tutorial/main/tmdb_5000_movies.csv').to_csv('movies.csv')

# pd.read_csv('https://raw.githubusercontent.com/4GeeksAcademy/k-nearest-neighbors-project-tutorial/main/tmdb_5000_credits.csv').to_csv('credits.csv')
df_movies = pd.read_csv('movies.csv')
df_credits = pd.read_csv('credits.csv')
pd.set_option('display.max_columns', None)



In [4]:
df_movies.head()

,Unnamed: 0,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [5]:
import sqlite3

# Crear o conectarse a la base de datos
conn = sqlite3.connect('movies_database.db')
cursor = conn.cursor()

In [6]:
df_movies.to_sql('movies', conn, if_exists='replace', index=False)

4803

In [7]:
df_credits.to_sql('credits', conn, if_exists='replace', index=False)

4803

In [8]:
cursor.execute('PRAGMA table_info(movies)')
columns = cursor.fetchall()
for col in columns:
    print(col)


(0, 'Unnamed: 0', 'INTEGER', 0, None, 0)
(1, 'budget', 'INTEGER', 0, None, 0)
(2, 'genres', 'TEXT', 0, None, 0)
(3, 'homepage', 'TEXT', 0, None, 0)
(4, 'id', 'INTEGER', 0, None, 0)
(5, 'keywords', 'TEXT', 0, None, 0)
(6, 'original_language', 'TEXT', 0, None, 0)
(7, 'original_title', 'TEXT', 0, None, 0)
(8, 'overview', 'TEXT', 0, None, 0)
(9, 'popularity', 'REAL', 0, None, 0)
(10, 'production_companies', 'TEXT', 0, None, 0)
(11, 'production_countries', 'TEXT', 0, None, 0)
(12, 'release_date', 'TEXT', 0, None, 0)
(13, 'revenue', 'INTEGER', 0, None, 0)
(14, 'runtime', 'REAL', 0, None, 0)
(15, 'spoken_languages', 'TEXT', 0, None, 0)
(16, 'status', 'TEXT', 0, None, 0)
(17, 'tagline', 'TEXT', 0, None, 0)
(18, 'title', 'TEXT', 0, None, 0)
(19, 'vote_average', 'REAL', 0, None, 0)
(20, 'vote_count', 'INTEGER', 0, None, 0)


In [9]:
cursor.execute('PRAGMA table_info(credits)')
columns = cursor.fetchall()
for col in columns:
    print(col)

(0, 'Unnamed: 0', 'INTEGER', 0, None, 0)
(1, 'movie_id', 'INTEGER', 0, None, 0)
(2, 'title', 'TEXT', 0, None, 0)
(3, 'cast', 'TEXT', 0, None, 0)
(4, 'crew', 'TEXT', 0, None, 0)


In [10]:
# Realizar la unión (JOIN) y crear una nueva tabla con el resultado
join_query = '''
CREATE TABLE IF NOT EXISTS unified  AS
SELECT m.id, m.title, m.overview, m.genres, m.keywords, c.cast, c.crew
FROM movies m
JOIN credits c ON m.title = c.title
'''

cursor.execute(join_query)
conn.commit()


In [11]:
df = pd.read_sql_query('SELECT * FROM unified', conn)
display(df)

,id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...","[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...","[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."
...,...,...,...,...,...,...,...
4804,9367,El Mariachi,El Mariachi just wants to play his guitar and ...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""id"": 5616, ""name"": ""united states\u2013mexi...","[{""cast_id"": 1, ""character"": ""El Mariachi"", ""c...","[{""credit_id"": ""52fe44eec3a36847f80b280b"", ""de..."
4805,72766,Newlyweds,A newlywed couple's honeymoon is upended by th...,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10749, ""...",[],"[{""cast_id"": 1, ""character"": ""Buzzy"", ""credit_...","[{""credit_id"": ""52fe487dc3a368484e0fb013"", ""de..."
4806,231617,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...","[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...","[{""id"": 248, ""name"": ""date""}, {""id"": 699, ""nam...","[{""cast_id"": 8, ""character"": ""Oliver O\u2019To...","[{""credit_id"": ""52fe4df3c3a36847f8275ecf"", ""de..."
4807,126186,Shanghai Calling,When ambitious New York attorney Sam is sent t...,[],[],"[{""cast_id"": 3, ""character"": ""Sam"", ""credit_id...","[{""credit_id"": ""52fe4ad9c3a368484e16a36b"", ""de..."


In [12]:
cursor.execute('PRAGMA table_info(unified)')
columns = cursor.fetchall()
for col in columns:
    print(col)

(0, 'id', 'INT', 0, None, 0)
(1, 'title', 'TEXT', 0, None, 0)
(2, 'overview', 'TEXT', 0, None, 0)
(3, 'genres', 'TEXT', 0, None, 0)
(4, 'keywords', 'TEXT', 0, None, 0)
(5, 'cast', 'TEXT', 0, None, 0)
(6, 'crew', 'TEXT', 0, None, 0)


In [13]:

def json_to_string(dataframe, column, term):
    import json
    column_list = []
    for index, row in dataframe.iterrows():
        row_data = []
        if row[column] and row[column] != '':
            list_ = json.loads(row[column])
            for dic in list_:
                row_data.append(dic[term])
            for i in range(len(row_data)):
                row_data[i] = row_data[i].replace(',', '').replace(' ', '')
            column_list.append(' '.join(row_data))
    return column_list




In [14]:
df

,id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...","[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...","[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."
...,...,...,...,...,...,...,...
4804,9367,El Mariachi,El Mariachi just wants to play his guitar and ...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""id"": 5616, ""name"": ""united states\u2013mexi...","[{""cast_id"": 1, ""character"": ""El Mariachi"", ""c...","[{""credit_id"": ""52fe44eec3a36847f80b280b"", ""de..."
4805,72766,Newlyweds,A newlywed couple's honeymoon is upended by th...,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10749, ""...",[],"[{""cast_id"": 1, ""character"": ""Buzzy"", ""credit_...","[{""credit_id"": ""52fe487dc3a368484e0fb013"", ""de..."
4806,231617,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...","[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...","[{""id"": 248, ""name"": ""date""}, {""id"": 699, ""nam...","[{""cast_id"": 8, ""character"": ""Oliver O\u2019To...","[{""credit_id"": ""52fe4df3c3a36847f8275ecf"", ""de..."
4807,126186,Shanghai Calling,When ambitious New York attorney Sam is sent t...,[],[],"[{""cast_id"": 3, ""character"": ""Sam"", ""credit_id...","[{""credit_id"": ""52fe4ad9c3a368484e16a36b"", ""de..."


In [15]:
df['genres'] = json_to_string(df, 'genres', 'name')
df['keywords'] = json_to_string(df, 'keywords', 'name')

In [16]:
import json

# Crear una lista vacía para almacenar los resultados
directors = []

for list_str in df['crew']:
    list_ = json.loads(list_str)
    director = ''
    for dic in list_:
        if dic['job'] == 'Director':
            director = dic['name'].replace(',', '').replace(' ', '')
            
    directors.append(''.join(director))

In [17]:
df['director'] = directors

df

,id,title,overview,genres,keywords,cast,crew,director
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...",Action Adventure Fantasy ScienceFiction,cultureclash future spacewar spacecolony socie...,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...",JamesCameron
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",Adventure Fantasy Action,ocean drugabuse exoticisland eastindiatradingc...,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de...",GoreVerbinski
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,Action Adventure Crime,spy basedonnovel secretagent sequel mi6 britis...,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de...",SamMendes
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,Action Crime Drama Thriller,dccomics crimefighter terrorist secretidentity...,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de...",ChristopherNolan
4,49529,John Carter,"John Carter is a war-weary, former military ca...",Action Adventure ScienceFiction,basedonnovel mars medallion spacetravel prince...,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de...",AndrewStanton
...,...,...,...,...,...,...,...,...
4804,9367,El Mariachi,El Mariachi just wants to play his guitar and ...,Action Crime Thriller,unitedstates–mexicobarrier legs arms paperknif...,"[{""cast_id"": 1, ""character"": ""El Mariachi"", ""c...","[{""credit_id"": ""52fe44eec3a36847f80b280b"", ""de...",RobertRodriguez
4805,72766,Newlyweds,A newlywed couple's honeymoon is upended by th...,Comedy Romance,,"[{""cast_id"": 1, ""character"": ""Buzzy"", ""credit_...","[{""credit_id"": ""52fe487dc3a368484e0fb013"", ""de...",EdwardBurns
4806,231617,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...",Comedy Drama Romance TVMovie,date loveatfirstsight narration investigation ...,"[{""cast_id"": 8, ""character"": ""Oliver O\u2019To...","[{""credit_id"": ""52fe4df3c3a36847f8275ecf"", ""de...",ScottSmith
4807,126186,Shanghai Calling,When ambitious New York attorney Sam is sent t...,,,"[{""cast_id"": 3, ""character"": ""Sam"", ""credit_id...","[{""credit_id"": ""52fe4ad9c3a368484e16a36b"", ""de...",DanielHsia


In [18]:
display(df['cast'])

0       [{"cast_id": 242, "character": "Jake Sully", "...
1       [{"cast_id": 4, "character": "Captain Jack Spa...
2       [{"cast_id": 1, "character": "James Bond", "cr...
3       [{"cast_id": 2, "character": "Bruce Wayne / Ba...
4       [{"cast_id": 5, "character": "John Carter", "c...
                              ...                        
4804    [{"cast_id": 1, "character": "El Mariachi", "c...
4805    [{"cast_id": 1, "character": "Buzzy", "credit_...
4806    [{"cast_id": 8, "character": "Oliver O\u2019To...
4807    [{"cast_id": 3, "character": "Sam", "credit_id...
4808    [{"cast_id": 3, "character": "Herself", "credi...
Name: cast, Length: 4809, dtype: object

In [19]:
import json
cast_list = []
for list_str in df['cast']:
    row_list = []
    list_ = json.loads(list_str)
    for dic in list_[:3]:
        row_list.append(dic['name'].replace(' ', '').replace(',', '').replace('. ', ''))
    cast_list.append(' '.join(row_list))

In [20]:
df['cast'] = cast_list
df

,id,title,overview,genres,keywords,cast,crew,director
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...",Action Adventure Fantasy ScienceFiction,cultureclash future spacewar spacecolony socie...,SamWorthington ZoeSaldana SigourneyWeaver,"[{""credit_id"": ""52fe48009251416c750aca23"", ""de...",JamesCameron
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",Adventure Fantasy Action,ocean drugabuse exoticisland eastindiatradingc...,JohnnyDepp OrlandoBloom KeiraKnightley,"[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de...",GoreVerbinski
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,Action Adventure Crime,spy basedonnovel secretagent sequel mi6 britis...,DanielCraig ChristophWaltz LéaSeydoux,"[{""credit_id"": ""54805967c3a36829b5002c41"", ""de...",SamMendes
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,Action Crime Drama Thriller,dccomics crimefighter terrorist secretidentity...,ChristianBale MichaelCaine GaryOldman,"[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de...",ChristopherNolan
4,49529,John Carter,"John Carter is a war-weary, former military ca...",Action Adventure ScienceFiction,basedonnovel mars medallion spacetravel prince...,TaylorKitsch LynnCollins SamanthaMorton,"[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de...",AndrewStanton
...,...,...,...,...,...,...,...,...
4804,9367,El Mariachi,El Mariachi just wants to play his guitar and ...,Action Crime Thriller,unitedstates–mexicobarrier legs arms paperknif...,CarlosGallardo JaimedeHoyos PeterMarquardt,"[{""credit_id"": ""52fe44eec3a36847f80b280b"", ""de...",RobertRodriguez
4805,72766,Newlyweds,A newlywed couple's honeymoon is upended by th...,Comedy Romance,,EdwardBurns KerryBishé MarshaDietlein,"[{""credit_id"": ""52fe487dc3a368484e0fb013"", ""de...",EdwardBurns
4806,231617,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...",Comedy Drama Romance TVMovie,date loveatfirstsight narration investigation ...,EricMabius KristinBooth CrystalLowe,"[{""credit_id"": ""52fe4df3c3a36847f8275ecf"", ""de...",ScottSmith
4807,126186,Shanghai Calling,When ambitious New York attorney Sam is sent t...,,,DanielHenney ElizaCoupe BillPaxton,"[{""credit_id"": ""52fe4ad9c3a368484e16a36b"", ""de...",DanielHsia


In [22]:
df_final = pd.DataFrame()
df_final['all_data'] = df['overview'] + ' ' + df['genres'] + ' ' + df['keywords'] + ' ' + df['cast'] + ' ' + df['director']
pd.set_option('display.max_colwidth', None)

In [23]:
df_final

,all_data
0,"In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. Action Adventure Fantasy ScienceFiction cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d SamWorthington ZoeSaldana SigourneyWeaver JamesCameron"
1,"Captain Barbossa, long believed to be dead, has come back to life and is headed to the edge of the Earth with Will Turner and Elizabeth Swann. But nothing is quite as it seems. Adventure Fantasy Action ocean drugabuse exoticisland eastindiatradingcompany loveofone'slife traitor shipwreck strongwoman ship alliance calypso afterlife fighter pirate swashbuckler aftercreditsstinger JohnnyDepp OrlandoBloom KeiraKnightley GoreVerbinski"
2,"A cryptic message from Bond’s past sends him on a trail to uncover a sinister organization. While M battles political forces to keep the secret service alive, Bond peels back the layers of deceit to reveal the terrible truth behind SPECTRE. Action Adventure Crime spy basedonnovel secretagent sequel mi6 britishsecretservice unitedkingdom DanielCraig ChristophWaltz LéaSeydoux SamMendes"
3,"Following the death of District Attorney Harvey Dent, Batman assumes responsibility for Dent's crimes to protect the late attorney's reputation and is subsequently hunted by the Gotham City Police Department. Eight years later, Batman encounters the mysterious Selina Kyle and the villainous Bane, a new terrorist leader who overwhelms Gotham's finest. The Dark Knight resurfaces to protect a city that has branded him an enemy. Action Crime Drama Thriller dccomics crimefighter terrorist secretidentity burglar hostagedrama timebomb gothamcity vigilante cover-up superhero villainess tragichero terrorism destruction catwoman catburglar imax flood criminalunderworld batman ChristianBale MichaelCaine GaryOldman ChristopherNolan"
4,"John Carter is a war-weary, former military captain who's inexplicably transported to the mysterious and exotic planet of Barsoom (Mars) and reluctantly becomes embroiled in an epic conflict. It's a world on the brink of collapse, and Carter rediscovers his humanity when he realizes the survival of Barsoom and its people rests in his hands. Action Adventure ScienceFiction basedonnovel mars medallion spacetravel princess alien steampunk martian escape edgarriceburroughs alienrace superhumanstrength marscivilization swordandplanet 19thcentury 3d TaylorKitsch LynnCollins SamanthaMorton AndrewStanton"
...,...
4804,"El Mariachi just wants to play his guitar and carry on the family tradition. Unfortunately, the town he tries to find work in has another visitor...a killer who carries his guns in a guitar case. The drug lord and his henchmen mistake El Mariachi for the killer, Azul, and chase him around town trying to kill him and get his guitar case. Action Crime Thriller unitedstates–mexicobarrier legs arms paperknife guitarcase CarlosGallardo JaimedeHoyos PeterMarquardt RobertRodriguez"
4805,A newlywed couple's honeymoon is upended by the arrivals of their respective sisters. Comedy Romance EdwardBurns KerryBishé MarshaDietlein EdwardBurns
4806,"""Signed, Sealed, Delivered"" introduces a dedicated quartet of civil servants in the Dead Letter Office of the U.S. Postal System who transform themselves into an elite team of lost-mail detectives. Their determination to deliver the seemingly undeliverable takes them out of the post office into an unpredictable world where letters and packages from the past save lives, solve crimes, reunite old loves, and change futures by arriving late, but always miraculously on time. Comedy Drama Romance TVMovie date loveatfirstsight narration investigation team postalworker EricMabius KristinBooth CrystalLowe ScottSmith"
4807,"When ambitious New York attorney Sam is sent to Shanghai on assignment, he imme